In [1]:
import pandas as pd
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
movies = pd.read_csv("archive/tmdb_5000_credits.csv")
credits = pd.read_csv("archive/tmdb_5000_movies.csv")

In [ ]:
#merge two data set columns wise
merged = pd.concat([movies,credits], axis=1)
merged.to_csv("merged_columns.csv", index=False)


In [2]:
mm = pd.read_csv("merged_columns.csv")

In [64]:
mm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   movie_id              4803 non-null   int64  
 2   title                 4803 non-null   object 
 3   cast                  4803 non-null   object 
 4   crew                  4803 non-null   object 
 5   budget                4803 non-null   int64  
 6   genres                4803 non-null   object 
 7   homepage              1712 non-null   object 
 8   id                    4803 non-null   int64  
 9   keywords              4803 non-null   object 
 10  original_language     4803 non-null   object 
 11  original_title        4803 non-null   object 
 12  overview              4800 non-null   object 
 13  popularity            4803 non-null   float64
 14  production_companies  4803 non-null   object 
 15  production_countries 

In [65]:
mm.isnull()

,index,movie_id,title,cast,crew,budget,genres,homepage,id,keywords,...,tagline,title.1,vote_average,vote_count,genre_names,keyword_names,combined,tags_string,cast_names,has_hero
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4799,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4800,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
4801,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [67]:
mm['cast'].info()
mm['genres'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 4803 entries, 0 to 4802
Series name: cast
Non-Null Count  Dtype 
--------------  ----- 
4803 non-null   object
dtypes: object(1)
memory usage: 37.7+ KB
<class 'pandas.core.series.Series'>
RangeIndex: 4803 entries, 0 to 4802
Series name: genres
Non-Null Count  Dtype 
--------------  ----- 
4803 non-null   object
dtypes: object(1)
memory usage: 37.7+ KB


In [79]:
mm['cast'].head(5)
mm['genres'].head(5)

0    [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
1    [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2    [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
3    [{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...
4    [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
Name: genres, dtype: object

In [77]:
mm['cast'].isnull().all()

np.False_

In [81]:
print(mm['cast'].head(2))

0    [{'cast_id': 242, 'character': 'Jake Sully', '...
1    [{'cast_id': 4, 'character': 'Captain Jack Spa...
Name: cast, dtype: object


In [3]:
mm.shape

(4803, 24)

In [4]:
mm .head(5)

,movie_id,title,cast,crew,budget,genres,homepage,id,keywords,original_language,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title.1,vote_average,vote_count
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,...,"[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [5]:
mm['genres'] = mm['genres'].apply(json.loads)
mm['keywords'] = mm['keywords'].apply(json.loads)

In [6]:
mm['genres'][0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [7]:
mm['genre_names'] = mm['genres'].apply(lambda x: [d['name'] for d in x])
mm['keyword_names'] = mm['keywords'].apply(lambda x: [d['name'] for d in x])


In [8]:
mm['combined'] = mm['genre_names'] + mm['keyword_names']
mm['tags_string'] = mm['combined'].apply(lambda x: ' '.join(x))

In [9]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(mm['tags_string'])
cosine_sim = cosine_similarity(tfidf_matrix)

In [10]:
mm = mm.reset_index()
indices = pd.Series(mm.index, index=mm['title'].str.lower()).drop_duplicates()

In [11]:
def recommend(title, top_n=5):
    title = title.lower()
    if title not in indices:
        return f" Movie '{title}' not found in the database."
    
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    movie_indices = [i[0] for i in sim_scores]
    
    return mm['title'].iloc[movie_indices].tolist()


In [91]:
user_movie = input("Enter a movie title: ")
recommendations = recommend(user_movie)

print("\nTop Recommended Movies:")

if isinstance(recommendations, list):
    for i, movie in enumerate(recommendations, 1):
        print(f"{i}. {movie}")
else:
    print(recommendations)
 


Enter a movie title:  Namastey London



Top Recommended Movies:
1. Housefull
2. Pandaemonium
3. Live-In Maid
4. The Circle
5. Bon voyage


In [27]:
mm['cast_names'] = mm['cast'].apply(lambda x: [d['name'] for d in x])

In [33]:
mm['cast'].head(1)

0    [{'cast_id': 242, 'character': 'Jake Sully', '...
Name: cast, dtype: object

In [34]:
mm['cast_names'] = mm['cast'].apply(lambda cast_list: [actor['name'] for actor in cast_list])

In [35]:
exploded = mm.explode('cast_names')

In [36]:
exploded['actor_lower'] = exploded['cast_names'].str.lower()

In [37]:
exploded_sorted = exploded.sort_values(by='vote_average', ascending=False)

In [46]:
mm['cast_names'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 4803 entries, 0 to 4802
Series name: cast_names
Non-Null Count  Dtype 
--------------  ----- 
4803 non-null   object
dtypes: object(1)
memory usage: 37.7+ KB


In [61]:
mm['cast_names'].head(-1)

0       [Sam Worthington, Zoe Saldana, Sigourney Weave...
1       [Johnny Depp, Orlando Bloom, Keira Knightley, ...
2       [Daniel Craig, Christoph Waltz, Léa Seydoux, R...
3       [Christian Bale, Michael Caine, Gary Oldman, A...
4       [Taylor Kitsch, Lynn Collins, Samantha Morton,...
                              ...                        
4797                                                   []
4798    [Carlos Gallardo, Jaime de Hoyos, Peter Marqua...
4799    [Edward Burns, Kerry Bishé, Marsha Dietlein, C...
4800    [Eric Mabius, Kristin Booth, Crystal Lowe, Geo...
4801    [Daniel Henney, Eliza Coupe, Bill Paxton, Alan...
Name: cast_names, Length: 4802, dtype: object

In [88]:
hero_input = input("Enter a hero/actor name: ").strip().lower()

top_movies = exploded_sorted[exploded_sorted['actor_lower'] == hero_input].head(5)

if top_movies.empty:
    print(f"\n No movies found for actor '{hero_input}'. Please check the name.")
else:
    print(f"\n Top Rated Movies of '{hero_input.title()}':\n")
    for i, row in top_movies.iterrows():
        print(f"{i+1}. {row['title']} (⭐ {row['vote_average']})")

Enter a hero/actor name:   Katrina Kaif



 Top Rated Movies of 'Katrina Kaif':

3158. Jab Tak Hai Jaan (⭐ 7.3)
3227. Namastey London (⭐ 6.6)
